# ICAIF 2024 금융-RAG 챌린지 기본 예제

이 노트북은 **ICAIF 2024 금융-RAG 챌린지**를 위한 **기본 예제**입니다. 이 챌린지의 목표는 금융 데이터를 위한 **Retrieval-Augmented Generation (RAG)** 시스템을 만드는 것입니다. 참가자는 대규모 코퍼스에서 관련 문서를 검색하고 사용자 Query에 대한 정확하고 상황에 맞는 응답을 제공하는 시스템을 개발해야 합니다.

---

## 시스템 구성 요소

기본 예제의 시스템은 두 가지 주요 구성 요소로 나뉩니다:

1. **검색**: 사용자 쿼리를 기반으로 대규모 금융 문서 코퍼스에서 관련 문서를 검색합니다.
2. **재정렬**: 검색된 문서의 순위를 다시 매겨 가장 관련성 높은 정보가 우선되도록 합니다.

---

## 모델 개요

이 베이스라인 노트북은 `SentenceTransformer`와 `CrossEncoder` 모델을 조합하여 다음 작업을 수행합니다:

- **검색 모델**은 쿼리와 문서를 임베딩으로 인코딩하는 역할을 담당합니다.
- **재정렬 모델**은 검색된 문서의 관련성을 평가하고 순서를 조정합니다.

이 예시에서는 **FinDER**라는 FinanceRAG 프로젝트의 7개 과제 중 하나를 사용합니다. 검색 모델로는 `intfloat/e5-large-v2`가 사용되며, 재정렬은 `cross-encoder/ms-marco-MiniLM-L-12-v2`를 통해 수행됩니다. 두 모델 모두 `sentence_transformers` 라이브러리에서 지원하는 다른 모델로 대체하여 성능을 실험해볼 수 있습니다.

---

## 목표

이 노트북의 목표는 참가자들이 챌린지를 위한 보다 **고급 솔루션**을 구축할 수 있는 **탄탄한 기반**을 제공하는 것입니다. 과제, 검색 모델 및 재정렬 모델을 필요에 따라 자유롭게 개발하세요!

---

## Repository Setup and Environment Configuration

GitHub 리포지토리 확인 [here](https://github.com/linq-rag/FinanceRAG).

아래와 같이 Github repository를 Clone하기:

### 1. Clone the repository:

```bash
git clone https://github.com/linq-rag/FinanceRAG.git
cd FinanceRAG
```

### 2. Set up the Python environment:

#### If using `venv` (Python 3.11 or higher required):

```bash
python3 -m venv .venv
source .venv/bin/activate  # On Windows use .venv\Scriptsctivate
pip install --upgrade pip
pip install -r requirements.txt
```

#### If using `conda`:

```bash
conda create -n financerag python=3.11
conda activate financerag
pip install -r requirements.txt
```

준비가 완료되었습니다!

In [3]:
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from sentence_transformers import CrossEncoder
import logging

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER
import torch

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
# Step 2: Initialize FinDER Task
# --------------------------
# In this baseline example, we are using the FinDER task, one of the seven available tasks in this project.
# If you want to use a different task, for example, 'OtherTask', you can change the task initialization as follows:
#
# Example:
# from financerag.tasks import OtherTask
# finder_task = OtherTask()
#
# For this baseline, we proceed with FinDER.
finder_task = FinDER()


INFO:financerag.common.loader:Loading Corpus...
INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsi

In [9]:
# Step 3: Initialize DenseRetriever model
# -------------------------------------
# Initialize the retrieval model using SentenceTransformers. This model will be responsible
# for encoding both the queries and documents into embeddings.
#
# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'BAAI/bge-large-en-v1.5', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='nvidia/NV-Embed-v2',
    query_prompt='query: ',
    doc_prompt='passage: ',
    device='cpu',
    trust_remote_code=True,
)

retrieval_model = DenseRetrieval(
    model=encoder_model,
)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: nvidia/NV-Embed-v2
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


In [10]:
# Step 4: Perform retrieval
# ---------------------
# Use the model to retrieve relevant documents for given queries.

retrieval_result = finder_task.retrieve(
    retriever=retrieval_model
)

# Print a portion of the retrieval results to verify the output.
print(f"Retrieved results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in retrieval_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.retrieval.dense:Encoding queries...
Batches:   0%|          | 0/4 [00:00<?, ?it/s]/usr/lib/python3.12/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
Batches: 100%|██████████| 4/4 [02:22<00:00, 35.52s/it]
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...
Batches:   0%|          | 0/217 [00:00<?, ?it/s]

: 

In [ ]:
# Step 5: Initialize CrossEncoder Reranker
# --------------------------------------
# The CrossEncoder model will be used to rerank the retrieved documents based on relevance.
#
# You can replace 'cross-encoder/ms-marco-MiniLM-L-12-v2' with any other model supported by CrossEncoder.
# For example: 'cross-encoder/ms-marco-TinyBERT-L-2', 'cross-encoder/stsb-roberta-large', etc.
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', device=device),
)

from FlagEmbedding import FlagLLMReranker
reranker = CrossEncoderReranker(model=FlagLLMReranker('BAAI/bge-reranker-v2-gemma'))

RuntimeError: Failed to import transformers.models.mllama.configuration_mllama because of the following error (look up to see its traceback):
No module named 'transformers.models.mllama.configuration_mllama'

In [14]:
# Step 6: Perform reranking
# -------------------------
# Rerank the top 100 retrieved documents using the CrossEncoder model.
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=500,  # Rerank the top 100 documents
    batch_size=256
)

# Print a portion of the reranking results to verify the output.
print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in reranking_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-500....
Batches:   0%|          | 0/85 [00:00<?, ?it/s]

Batches: 100%|██████████| 85/85 [00:36<00:00,  2.32it/s]


Reranking results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230254, Score = 6.622740745544434
  Document 2: Document ID = MSFT20230233, Score = 6.088944435119629
  Document 3: Document ID = MSFT20230230, Score = 5.898367404937744
  Document 4: Document ID = MSFT20230236, Score = 5.747088432312012
  Document 5: Document ID = MSFT20230216, Score = 5.488572120666504


In [7]:
# Step 7: Save results
# -------------------
# Save the results to the specified output directory as a CSV file.
output_dir = './results'
finder_task.save_results(output_dir=output_dir)

# Confirm the results have been saved.
print(f"Results have been saved to {output_dir}/FinDER/results.csv")


INFO:financerag.tasks.BaseTask:Output directory set to: ./results/FinDER
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results/FinDER/results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results/FinDER/results.csv


Results have been saved to ./results/FinDER/results.csv
